In [ ]:
import rospy
import tf
import json
from moveit_msgs.srv import GetPositionIK, GetPositionIKRequest
from geometry_msgs.msg import PoseStamped
from cs_golf.robot import Robot
from cs_golf.trajectories import trapezoidal_speed_trajectory
from copy import deepcopy
from cs_golf.transformations import quat_rotate, multiply_transform, inverse_transform, list_to_pose_stamped
from moveit_msgs.msg import RobotTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint

In [ ]:
rospy.init_node("putter_rotations")

In [ ]:
tfl = tf.TransformListener()

In [ ]:
tfb = tf.TransformBroadcaster()

In [ ]:
# rosrun tf static_transform_publisher -0.385 -0.396 0.02 -0.604 -0.405 -0.601 0.331 iiwa_link_ee putter 50
# roslaunch iiwa_moveit demo.launch driver:=true

In [ ]:
DISTANCE_TO_CENTRE = 0.12    # Run-up
HEIGHT_ANGLE_SHOULDER = 0.75 
NUM_JOINTS = 7

In [ ]:
world_P_centre = [[0.0 + DISTANCE_TO_CENTRE, -0.55, 0.03], [0, 0, 0, 1]]

In [ ]:
gripper_T_putter = [[-0.385, -0.396, 0.02], [-0.604, -0.405, -0.601, 0.331]]
putter_T_gripper = inverse_transform(gripper_T_putter)

In [ ]:
ik = rospy.ServiceProxy("/iiwa/compute_ik", GetPositionIK)

In [ ]:
robot = Robot()

In [ ]:
NUM_ROTATIONS = 100
NUM_POINTS_IN_TRAJ = 20

# Seed is poses["init"]
last_ik_seed_angle = robot.current_state
last_ik_seed_angle.joint_state.position = [-0.11969436376084419, -0.494854232671294, -1.3905943305659678,
                                           -1.764423531111349, 1.0661701451509533, 1.511239882188521, 0.3787167269893851]

#rospy.sleep(3)
trajectories = []
diffs = []
for rot in range(NUM_ROTATIONS):
    # Compute each putter angle between [0, 0.7[ rad
    angle = 0 + rot*0.55/NUM_ROTATIONS
    rotation = tf.transformations.quaternion_about_axis(angle, (0, 0, 1))
    trajectory = {"angle": angle, "points": []}
    
    world_P_centre_angled = [world_P_centre[0], rotation]
    #tfb.sendTransform(world_P_centre_angled[0], world_P_centre_angled[1], rospy.Time.now(), "centre_rot", "world")
    
    world_P_shoulder = deepcopy(world_P_centre_angled)
    world_P_shoulder[0][2] += HEIGHT_ANGLE_SHOULDER
    
    # rotation movement of pi/4 (= 70/100)
    last_ik_seed = last_ik_seed_angle
    CIRCLE_RANGE = 80
    for rot2 in range(CIRCLE_RANGE):
        # Publish "shoulder" in this loop so that it's always available
        tfb.sendTransform(world_P_shoulder[0], world_P_shoulder[1], rospy.Time.now(), "shoulder", "world")
        angle2 = rot2 / 100. + 0.002*CIRCLE_RANGE
        reverse_angle2 = 0.7 - angle2
        rotation2 = tf.transformations.quaternion_about_axis(reverse_angle2, (0, 1, 0))
        shoulder_T_putter_rot = quat_rotate(rotation2, [0, 0, -HEIGHT_ANGLE_SHOULDER])
        shoulder_T_putter = [shoulder_T_putter_rot, rotation2]
        tfb.sendTransform(shoulder_T_putter[0], shoulder_T_putter[1], rospy.Time.now(), "putter2", "shoulder")
        #rospy.sleep(0.01)

        # Get end effector pose for each angle
        shoulder_P_gripper = multiply_transform(shoulder_T_putter, putter_T_gripper)
        world_P_gripper = multiply_transform(world_P_shoulder, shoulder_P_gripper)

        tfb.sendTransform(world_P_gripper[0], world_P_gripper[1], rospy.Time.now(), "gripper", "world")
        ps_eef = list_to_pose_stamped(world_P_gripper, frame_id="world")
        
        # Get IK for the init pose       
        ik_ok = False
        for attempt in range(50):
            req = GetPositionIKRequest()
            req.ik_request.group_name = "manipulator"
            req.ik_request.ik_link_name = "iiwa_link_ee"
            req.ik_request.pose_stamped = ps_eef
            req.ik_request.robot_state = last_ik_seed_angle
            result = ik(req)
            if result.error_code.val != -31:
                if rot2 in [0, 1]:
                    smoothed_positions = result.solution.joint_state.position
                else:
                    previous = trajectory['points'][-1]
                    smoothed_positions = [0.8*v + 0.2*previous[i] for i, v in enumerate(result.solution.joint_state.position)]
                diff = sum([abs(last_ik_seed.joint_state.position[i] - smoothed_positions[i]) for i in range(NUM_JOINTS)])
                #print("Rotation", rot, rot2, "diff", round(diff, 4))
                if diff < 0.05 or rot2 == 0 and diff < 0.5 or rot == 0:
                    if rot2 == 0:
                        diffs.append(diff)
                    ik_ok = True
                    break
                    #print("Acceleration of rotation", rot, "is too high", diff, "OK" if rot2==0 else "")
                    #if rot not in non_valid_accel:
                    #    non_valid_accel[rot] = [diff]
                    #else:
                    #    non_valid_accel[rot].append(diff)
        if ik_ok:
            last_ik_seed = result.solution
            last_ik_seed.joint_state.position = smoothed_positions
            #rospy.sleep(0.02)
            trajectory["points"].append(result.solution.joint_state.position)
            if rot2 == 0:
                last_ik_seed_angle = last_ik_seed
            #print([int(v*180/3.1459) for v in result.solution.joint_state.position])
        else:
            print("No IK solution for", rot, "angle", angle, "rot2", rot2, "diff", diff)
            print([int(v*180/3.1459) for v in result.solution.joint_state.position])
            #raise RuntimeError()
    trajectories.append(trajectory)

In [ ]:
t = trajectories[71]
rt = RobotTrajectory()
rt.joint_trajectory.joint_names = last_ik_seed.joint_state.name
for point in t["points"]:
    jtp = JointTrajectoryPoint()
    jtp.positions = point
    rt.joint_trajectory.points.append(jtp)
robot.display(rt)
rospy.sleep(0.5)

In [ ]:
with open("../config/motions.json", "w") as f:
    json.dump({"joints": last_ik_seed.joint_state.name, "trajectories": trajectories}, f)

In [ ]:
max(diffs)

In [ ]:
min(diffs)

In [ ]:
diffs

In [ ]:
len(diffs)